In [1]:
import os
import pycrfsuite
import pickle
from sklearn_crfsuite import metrics
from sklearn.metrics import classification_report
from tqdm import tqdm

In [2]:
class MyClass():
    def __init__(self, param):
        self.param = param

def load_object(filename):
    try:
        with open(filename, "rb") as f:
            return pickle.load(f)

    except Exception as ex:
        print("Error during unpickling object (Possibly unsupported):", ex)

In [3]:
def test_crf_model(path_to_model, path_to_validation_batch):
    y_true = []
    y_pred = []

    tagger = pycrfsuite.Tagger()
    tagger.open(path_to_model)

    for file_name in tqdm(sorted(os.listdir(path_to_validation_batch + "/sequence_multimodal"))):

        multimodal_obj = load_object(path_to_validation_batch + "/sequence_multimodal/" + str(file_name))
        multimodal_seq = multimodal_obj.param

        sequential_information_obj = load_object(path_to_validation_batch + "/sequence_sequential_information/sequence_sequential_information_" + file_name.split('multimodal_')[1])
        sequential_information_seq = sequential_information_obj.param

        feature_seq = []

        for idx in range(len(multimodal_seq)):
            feature_seq.append({**multimodal_seq[idx], **sequential_information_seq[idx]})

        tagger.set(feature_seq)
        labels = tagger.tag()
        y_pred.append(labels)

        file_name_label = path_to_validation_batch + "/sequence_labels/sequence_labels_" + file_name.split('multimodal_')[1]
        label_obj = load_object(file_name_label)
        label_seq = label_obj.param

        for idx in range(len(label_seq)):
            label_seq[idx] = str(label_seq[idx])

        y_true.append(label_seq)

    flattened_y_true = []
    flattened_y_pred = []

    for idx in range(len(y_true)):
        for idx2 in range(len(y_true[idx])):
            flattened_y_true.append(y_true[idx][idx2])
            flattened_y_pred.append(y_pred[idx][idx2])

    print("Classification report on validation batch (" + path_to_validation_batch + ") for model \"" + path_to_model + "\": ")
    print(classification_report(flattened_y_true, flattened_y_pred, digits = 6))

In [ ]:
test_crf_model('/path_to_model_to_be_evaluated/model.crfsuite', '/path_to_validation_data_folder')